# ATAC-seq clustering and chromatin analysis

## 1. Cell type clustering 

**Goal**: Visualize sample relationships by signal similarity

**Steps**:
- Load & transpose filtered matrix
- merge with metadata (Sorted Populations file)
- run PCA/UMAP/t-SNE
- plot samples colored by cell type

In [2]:
# Load the filtered ATAC file with abT and Tact cells

import pandas as pd

peak_matrix = pd.read_csv("ATAC-seq/filtered_ATAC_abT_Tact_Stem.csv")
print(peak_matrix.shape)
peak_matrix.head()

(512595, 37)


,ImmGenATAC1219.peakID,chrom,Summit,mm10.60way.phastCons_scores,_-log10_bestPvalue,Included.in.systematic.analysis,TSS,genes.within.100Kb,preT.DN1.Th,preT.DN2a.Th,...,T8.Tcm.LCMV.d180.Sp,T8.Tem.LCMV.d180.Sp,NKT.Sp,NKT.Sp.LPS.3hr,NKT.Sp.LPS.18hr,NKT.Sp.LPS.3d,LTHSC.34-.BM,LTHSC.34+.BM,STHSC.150-.BM,MPP4.135+.BM
0,ImmGenATAC1219.peak_1,chr1,3020786,0.00,0.56,NaN,NaN,NaN,0.11,0.77,...,0.71,2.78,0.56,0.11,0.64,1.14,0.41,0.71,0.90,0.11
1,ImmGenATAC1219.peak_2,chr1,3087226,0.00,0.50,NaN,NaN,NaN,0.77,0.77,...,0.94,3.56,0.98,2.12,0.12,0.11,0.41,1.64,0.90,0.83
2,ImmGenATAC1219.peak_3,chr1,3120109,0.07,10.80,1.0,NaN,NaN,0.40,0.77,...,1.32,2.79,0.56,1.72,0.12,0.11,2.36,0.10,0.90,0.11
3,ImmGenATAC1219.peak_4,chr1,3121485,0.15,3.02,1.0,NaN,NaN,0.46,0.11,...,3.41,0.11,2.86,0.11,0.64,2.20,0.41,0.10,0.11,0.11
4,ImmGenATAC1219.peak_5,chr1,3372787,0.03,1.31,NaN,NaN,NaN,0.77,0.84,...,0.94,0.11,1.59,0.51,0.12,0.53,0.41,0.10,0.11,0.11


 We want to reduce the dimensionalityt of samples in this file based on peaks profiles for the clustering analysis. For that, we need to transpose the matrix to samples x peaks.

In [5]:
# Transposing the matrix

import pandas as pd
sample_matrix = peak_matrix.T
sample_matrix.head
type(sample_matrix)

pandas.core.frame.DataFrame

Now we a data frame with one row per sample and one column per peak feature. Now, we want to merge it with the metadata, so the file with the sorted abT and Tact cell populations. This will allow us to connect the information we get about the peaks to the biological background when we run the PCA / UMAP / t-SNE tests. We will be ablet to later label samples and color-code them for each cell type, respectively.